In [3]:
!pip install pyspark numpy
!pip install mpi4py


x: [92 45 51 34 32  8 10 90 75 56 85 41 55 81 14 67]
y: [98 87 30 67 73 79 51 58 65 16 70 84  9 50 68 96]
Final result: [190, 132, 81, 101, 105, 87, 61, 148, 140, 72, 155, 125, 64, 131, 82, 163]
Total execution time: 0.0008167000000014468


In [4]:
from mpi4py import MPI
import numpy as np

# Khởi tạo MPI
comm = MPI.COMM_WORLD

# Lấy thứ hạng của communicator
rank = comm.Get_rank()

# Kích thước của quá trình
size = comm.Get_size()

# Khởi tạo biến
N = int(16)
sum_xy = []
execution_time = 0

# Quá trình chính
if rank != 0:
    # Bắt đầu thời gian
    start = MPI.Wtime()

    # Nhận các mảng con từ quá trình chính
    a = comm.recv(source=0)
    b = comm.recv(source=0)

    # Tính tổng các mảng con
    sum_xy = a + b

    # Gửi tổng trở lại quá trình chính
    comm.send(sum_xy, dest=0, tag=1)

    # Dừng thời gian
    end = MPI.Wtime()

    # Gửi thời gian thực thi đến quá trình chính
    comm.send(end - start, dest=0, tag=2)

    # In thời gian thực thi
    print("Thời gian thực thi của worker ", rank, "là:", end - start)

# Quá trình con
else:
    # Bắt đầu thời gian
    start = MPI.Wtime()

    # Tạo các mảng ngẫu nhiên x và y
    x = np.random.randint(100, size=N)
    y = np.random.randint(100, size=N)

    # Chia các mảng thành các mảng con
    split_x = np.array_split(x, size)
    split_y = np.array_split(y, size)

    # Gửi các mảng con đến các quá trình con
    for worker in range(1, size):
        comm.send(split_x[worker], dest=worker)
        comm.send(split_y[worker], dest=worker)

    # Nhận tổng từ các quá trình con
    for worker in range(1, size):
        sum_xy.extend(list(comm.recv(source=worker, tag=1)))

        # Nhận thời gian thực thi từ các quá trình con
        execution_time += comm.recv(source=worker, tag=2)

    # Dừng thời gian
    end = MPI.Wtime()

    # Cập nhật thời gian thực thi
    execution_time += (end - start)

# In kết quả
print("x:", x)
print("y:", y)
print("Kết quả cuối cùng:", sum_xy)
print("Tổng thời gian thực thi:", execution_time)

x: [ 0 44 17  0  9 41  4  6 83 50  1 67 78 56 57  9]
y: [51 77 94 14 37 80 79 97 97 92 46 77 87 72 63 67]
Kết quả cuối cùng: []
Tổng thời gian thực thi: 0.00031771700000149394


In [6]:
# Khởi tạo thư viện
from mpi4py import MPI
import numpy as np

# Khởi tạo MPI
comm = MPI.COMM_WORLD

# Lấy thứ hạng của worker
rank = comm.Get_rank()

# Lấy kích thước của quá trình
size = comm.Get_size()

# Khởi tạo biến
N = int(1e4)
average = None
average_x = []
execution_time = 0

# Quá trình chính
if rank != 0:
    start = MPI.Wtime()

    # Nhận mảng con từ quá trình chính
    a = comm.recv(source=0)

    # Gửi tổng của mảng con đến quá trình chính
    comm.send(np.sum(a) / len(a), dest=0, tag=1)

    end = MPI.Wtime()

    # Gửi thời gian thực thi đến quá trình chính
    comm.send(end - start, dest=0, tag=2)

    # In thời gian thực thi
    print("Thời gian thực thi của worker ", rank, "là:", end - start)

# Quá trình con
else:
    start = MPI.Wtime()

    # Tạo mảng ngẫu nhiên x
    x = np.random.randint(100, size=N)

    # Chia mảng x thành các mảng con
    split_x = np.array_split(x, size)

    # Nếu là quá trình chính
    if rank == 0:
        average_x.append(np.sum(split_x[rank]) / len(split_x[rank]))

    # Nếu là quá trình con
    else:
        comm.send(split_x[rank], dest=rank)

        average_x.append(comm.recv(source=rank, tag=1))

        execution_time += comm.recv(source=rank, tag=2)

    # Tính trung bình của các mảng con
    average = np.average(average_x)

    end = MPI.Wtime()

    execution_time += (end - start)

    # In thời gian thực thi
    print("Thời gian thực thi của worker ", rank, "là:", end - start)

    # In tổng thời gian thực thi
    print("Tổng thời gian thực thi:", np.sum(execution_time))

    # In mảng x
    print("\nX:", x)

    # In trung bình
    print("Trung bình:", average)

Thời gian thực thi của worker  0 là: 0.0010160829999676935
Tổng thời gian thực thi: 0.0010160829999676935

X: [82 38 89 ... 80 92 38]
Trung bình: 49.2013


In [9]:
# Khởi tạo thư viện
from mpi4py import MPI
import numpy as np

# Khởi tạo MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Khởi tạo biến
N = int(1e4)
VecMult = []
execution_time = 0

# Quá trình chính
if rank != 0:
    start = MPI.Wtime()

    # Nhận ma trận A và vector b từ quá trình chính
    A = comm.recv(source=0)
    b = comm.recv(source=0)

    # Tính tích vô hướng của A và b
    out = np.matmul(A, b)

    # Gửi tích vô hướng đến quá trình chính
    comm.send(out, dest=0, tag=1)

    end = MPI.Wtime()

    # Gửi thời gian thực thi đến quá trình chính
    comm.send(end - start, dest=0, tag=2)

    # In thời gian thực thi
    print("Thời gian thực thi của worker ", rank, "là:", end - start)

# Quá trình con
else:
    start = MPI.Wtime()

    # Tạo ma trận ngẫu nhiên A và vector ngẫu nhiên b
    A = np.random.randint(100, size=(N, N))
    b = np.random.randint(100, size=(N, 1))

    # Chia ma trận A thành các ma trận con
    split_A = np.array_split(A, size)

    # Nếu là quá trình chính
    if rank == 0:
        # Tính tích vô hướng của split_A[rank] và b
        out = np.matmul(split_A[rank], b)

        # Thêm tích vô hướng vào danh sách VecMult
        VecMult.extend(out.flatten().tolist())

    # Nếu là quá trình con
    else:
        # Gửi split_A[rank] và b đến quá trình con rank
        comm.send(split_A[rank], dest=rank)
        comm.send(b, dest=rank)

        # Nhận tích vô hướng từ quá trình con rank
        out = comm.recv(source=rank, tag=1)

        # Thêm tích vô hướng vào danh sách VecMult
        VecMult.extend(out.flatten().tolist())

        # Cập nhật thời gian thực thi
        execution_time += comm.recv(source=rank, tag=2)

    end = MPI.Wtime()

    # Cập nhật thời gian thực thi
    execution_time += (end - start)

    # In thời gian thực thi
    print("Thời gian thực thi của worker ", rank, "là:", end - start)

    # In tổng thời gian thực thi
    print("Tổng thời gian thực thi:", np.sum(execution_time))

    # In ma trận A và vector b
    print("\nA:", A)
    print("b:", b)

    # In tích vô hướng của A và b
    print("Tích vô hướng:", VecMult)

    # In độ dài của danh sách VecMult
    print(len(VecMult))

Thời gian thực thi của worker  0 là: 1.4638594069999726
Tổng thời gian thực thi: 1.4638594069999726

A: [[62 90 50 ... 44 49 89]
 [54 37 66 ...  1 93 43]
 [41 32 36 ... 44 45 33]
 ...
 [11 76 87 ... 55 12  9]
 [ 6  3 69 ... 68 83 61]
 [77 14 21 ... 27 86 88]]
b: [[39]
 [77]
 [24]
 ...
 [11]
 [71]
 [93]]
Tích vô hướng: [24463414, 24698563, 24474952, 24318282, 24470296, 24481800, 24468905, 24718504, 24472656, 24328010, 24597703, 24514628, 24536878, 24430506, 24659202, 24146507, 23936092, 24497173, 24248448, 24400061, 24259016, 24560962, 24338419, 24231996, 24421700, 24583423, 24434197, 24371446, 24289994, 24026634, 24200391, 24518038, 24522553, 24804862, 24662292, 24540557, 24221777, 24380217, 24461810, 24765674, 24407259, 24311234, 24338302, 24514933, 24362732, 24474825, 24342956, 24481389, 24603904, 24276622, 24467459, 24640961, 24410758, 24406219, 24141125, 24388418, 24283125, 24460610, 24353280, 24360063, 24686665, 24417809, 24609974, 24280213, 24112380, 24475744, 24597253, 24269882,

In [10]:
# Khởi tạo thư viện
from mpi4py import MPI
import numpy as np

# Khởi tạo MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

# Khởi tạo ma trận
A = None
B = None
N = int(1e2)
C = np.zeros((N, N))

# Quá trình chính
if rank != 0:
    B = np.empty((N, N))
else:
    A = np.random.rand(N, N)
    B = np.random.rand(N, N)

# Nhận biến
Arecv = np.empty((int(N / size), N))
start = MPI.Wtime()

# Phân chia dữ liệu cho tất cả các worker
comm.Scatter(A, Arecv, root=0)

# Gửi một bản sao của vector đến tất cả các worker
comm.Bcast(B, root=0)

# Thu thập kết quả từ tất cả các worker
comm.Gather(np.matmul(Arecv, B), C, root=0)

end = MPI.Wtime()
print("Thời gian thực thi của worker ", rank, "là:", end - start)

Thời gian thực thi của worker  0 là: 0.005475418000060017
